In [ ]:
import numpy as np
import theano as theano
import theano.tensor as T
import time
import operator
from utils import *
from lstm_theano import *
from gru_theano import *

In [ ]:
# Load data
from utils import load_and_proprocess_data
VOCABULARY_SIZE = 8000
X_train, y_train, word_to_index, index_to_word = load_and_proprocess_data(VOCABULARY_SIZE)

Reading CSV file...
Parsed 396962 sentences.

In [ ]:
# Do a gradient check
np.random.seed(0)
model = GRUTheano(100, 10)
gradient_check_theano(model, [0,1,2,3], [1,2,3,4])

In [ ]:
# Load parameters of pre-trained model
model = load_model_parameters_theano('./data/GRUTheano-80-8000-27.npz', GRUTheano)

# Build model and train

# LEARNING_RATE = 1e-4
# NEPOCH = 50
# HIDDEN_DIM = 80

# model = GRUTheano(VOCABULARY_SIZE, hidden_dim=HIDDEN_DIM)

# t1 = time.time()
# model.sgd_step(X_train[10], y_train[10], LEARNING_RATE)
# t2 = time.time()
# print "SGD Step time: %f milliseconds" % ((t2 - t1) * 1000.)

# train_with_sgd(model, X_train[:1000], y_train[:1000], LEARNING_RATE, NEPOCH, evaluate_loss_after=1)
# save_model_parameters_theano('data/mymodel.npz', model)

In [ ]:
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
unknown_token = "UNKNOWN_TOKEN"
    
def generate_sentence(model):
    # We start the sentence with the start token and a random word
    new_sentence = [word_to_index[sentence_start_token]]
    prob = 1
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)
        samples = np.random.multinomial(1, next_word_probs[-1])
        sampled_word = np.argmax(samples)
        prob = prob * next_word_probs[-1][sampled_word]
        # Discard this sentence if we sample an unknown word
        if sampled_word == word_to_index[unknown_token]:
           return [1,[]]
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return [prob, sentence_str]
 
num_sentences = 50
senten_min_length = 7
sentences = []
    
for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        prob, sent = generate_sentence(model)
    print "%f %s" % (prob, " ".join(sent))
    sentences.append([prob, sent])